# Amazon Bedrock Knowledge Bases를 활용한 메타데이터 필터링
이 노트북은 Amazon Bedrock Knowledge Bases의 'metadata filtering' 기능을 살펴보는 샘플 코드를 제공합니다.

메타데이터 필터링 기능을 사용하면 벡터 스토어에서 검색 전에 필터를 적용해 검색 결과를 향상시킬 수 있습니다. 
자세한 내용은 이 [블로그](https://aws.amazon.com/blogs/machine-learning/amazon-bedrock-knowledge-bases-now-supports-metadata-filtering-to-improve-retrieval-accuracy/)를 참고하세요.

## 1. 필요한 라이브러리 불러오기
먼저 필요한 패키지를 설치합니다.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import botocore
botocore.__version__

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name = 'metadata-filtering-kb'
knowledge_base_description = "Knowledge Base metadata filtering."
bucket_name = f'{knowledge_base_name}-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

# Define data sources
data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2 - 고정 청킹 전략으로 Knowledge Base 생성
비디오 게임 데이터를 CSV 형식으로 저장할 [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/)를 먼저 생성해 보겠습니다. Knowledge Bases는 [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise]() 및 [MongoDB Atlas]() 등 다양한 벡터 데이터베이스와 통합할 수 있습니다. 이 예제에서는 Knowledge Base를 Amazon OpenSearch Serverless와 연동합니다. 이를 위해 Knowledge Base와 필요한 사전 준비 작업을 모두 생성해 주는 헬퍼 클래스 `BedrockKnowledgeBase`를 사용합니다:
1. IAM 역할 및 정책
2. S3 버킷
3. Amazon OpenSearch Serverless 암호화, 네트워크 및 데이터 접근 정책
4. Amazon OpenSearch Serverless 컬렉션
5. Amazon OpenSearch Serverless 벡터 인덱스
6. Knowledge Base
7. Knowledge Base 데이터 소스

고정 청킹 전략을 사용해 Knowledge Base를 생성합니다. 

아래 매개변수 값을 변경하면 다른 청킹 전략을 선택할 수 있습니다: 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [ ]:
knowledge_base_metadata = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source, 
    chunking_strategy = "FIXED_SIZE", 
    suffix = suffix
)

### 2.1 비디오 게임 데이터 세트를 다운로드해 Amazon S3에 업로드하기

Knowledge Base를 만들었으니 `video_games` 데이터 세트로 내용을 채워 보겠습니다. 이 데이터는 [여기](https://aws-blogs-artifacts-public.s3.amazonaws.com/ML-16482/30_generated_video_game_records.zip)에서 다운로드하며, 각 비디오 게임의 제목, 설명, 장르, 연도, 퍼블리셔, 점수 정보를 포함한 가상의 비디오 게임 데이터입니다.

In [ ]:
import os
import zipfile

# Download the zip file
!wget https://aws-blogs-artifacts-public.s3.amazonaws.com/ML-16482/30_generated_video_game_records.zip --no-check-certificate

# Unzip the file content - This data will get unzipped into a folder name 'video_game'
with zipfile.ZipFile('./30_generated_video_game_records.zip', 'r') as zipf:
    csv_files = [x for x in zipf.infolist() if not x.filename.startswith('__MACOSX/') and x.filename.endswith('.csv')]
    for csv_file in csv_files:
        zipf.extract(csv_file, './')

#remove original zip file
# os.remove('./30_generated_video_game_records.zip')

`video_game` 폴더에 있는 비디오 게임 데이터를 S3에 업로드합니다.

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                if not file.startswith('.DS_Store'):
                    file_to_upload = os.path.join(root,file)
                    print(f"uploading file {file_to_upload} to {bucket_name}")
                    s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("video_game", bucket_name)

이제 ingestion 작업을 시작합니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_metadata.start_ingestion_job()

마지막으로 나중에 솔루션을 테스트할 수 있도록 Knowledge Base ID를 저장합니다.

In [ ]:
kb_id_metadata = knowledge_base_metadata.get_knowledge_base_id()

### 2.2 메타데이터 없이 Retrieve and Generate API로 Knowledge Base 질의

[**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) API를 사용해 Knowledge Base를 테스트해 보겠습니다. 이 API는 Bedrock이 Knowledge Base에서 필요한 참조를 검색한 뒤 Bedrock의 Foundation Model을 사용해 최종 답변을 생성합니다.

```
query = "A strategy game with cool graphic with score of 9.0"
```

예상 결과:
    * Fantasy Kingdoms: Chronicles of Eldoria is a strategy RPG game with a score of 9.0.

In [ ]:
query = "A strategy game with cool graphic with score of 9.0"

In [ ]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_metadata,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

pprint.pp(response['output']['text'])

#### 2.3 적재용 메타데이터 준비

In [ ]:
import csv
import json
import pandas as pd

def generate_matadata(data_dir , metadata_fields):
    # Define the metadata attributes
    metadata_attributes = metadata_fields

    # Loop through all CSV files in the directory
    for filename in os.listdir(data_dir):
        filename= f'{data_dir}/{filename}'
        if filename.endswith(".csv"):
            # Read the CSV file
            df = pd.read_csv(filename)
            df["Id"] = [os.path.basename(filename)]
            
            # Extract the metadata attributes
            metadata = {k:v[0] for k,v in df[metadata_attributes].to_dict(orient='list').items()}
            # reorder the keys
            metadata = {key: metadata[key] for key in metadata_attributes}
            
            # Create a JSON object
            json_data = {"metadataAttributes": metadata}
            
            
            # Write the JSON object to a file
            with open(f"{filename.replace('.csv', '.csv.metadata.json')}", "w") as f:
                json.dump(json_data, f)

In [ ]:
data_dir = './video_game'
metadata_fields = ["Id", "genres", "year", "publisher", "score"]

generate_matadata(data_dir, metadata_fields)

In [ ]:
# upload metadata file to S3
upload_directory("video_game", bucket_name)

In [ ]:
# delete metadata files from local
data_dir = './video_game'
for filename in os.listdir(data_dir):
    filename= f'{data_dir}/{filename}'
    if filename.endswith(".csv.metadata.json"):
        os.remove(filename)

이제 ingestion 작업을 시작합니다. 이미 고정 청킹에서 사용한 것과 동일한 문서를 사용하므로, S3 버킷에 문서를 업로드하는 단계는 건너뜁니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_metadata.start_ingestion_job()

### 2.4 메타데이터와 함께 Retrieve and Generate API로 Knowledge Base 질의

필터를 생성합니다.

In [ ]:
one_group_filter= {
    "andAll": [
        {
            "equals": {
                "key": "genres",
                "value": "Strategy"
            }
        },
        {
            "greaterThanOrEquals": {
                "key": "score",
                "value": 9.0
            }
        }
    ]
}

생성한 필터를 [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html)의 `retrievalConfiguration`에 전달합니다.

In [ ]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id_metadata,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5,
                    "filter": one_group_filter
                } 
            }
        }
    }
)

print(response['output']['text'])

이처럼 Retrieve and Generate API를 사용하면 최종 응답을 바로 받을 수 있습니다. 이제 `RetrieveAndGenerate` API에서 제공하는 인용 정보를 살펴보고, 응답을 생성할 때 모델이 반환하는 검색된 청크와 인용을 확인해 보겠습니다. 질의와 함께 적절한 컨텍스트를 Foundation Model에 제공하면 고품질 응답이 생성될 가능성이 크게 높아집니다. 

In [ ]:
# response_metadata = response['citations'][0]['retrievedReferences']
# print("# of citations or chunks used to generate the response: ", len(response_metadata))
# def citations_rag_print(response_ret):
# #structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
#     for num,chunk in enumerate(response_ret,1):
#         print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
#         print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
#         print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

# citations_rag_print(response_metadata)

### 정리
이 노트북에서 만든 리소스를 삭제하려면 아래 셀의 주석을 해제한 뒤 실행하세요. `03-advanced-concepts` 섹션의 `dynamic-metadata-filtering` 노트북을 실행할 계획이라면, 해당 노트북을 실행한 후 돌아와 리소스를 정리하세요. 

In [ ]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

In [ ]:
# print("===============================Knowledge base==============================")
knowledge_base_metadata.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)